In [ ]:
https://satijalab.org/seurat/articles/seurat5_integration

Seurat v5 propose une analyse intégrative simplifiée via la fonction IntegrateLayers() qui facilite l’intégration de plusieurs jeux de données de manière cohérente, en s’assurant que les cellules partagées entre les lots sont intégrées dans un espace de faible dimension. Ce processus est utile lorsqu’on travaille avec plusieurs jeux de données (par exemple, issus de différentes expériences ou traitements) et qu’on souhaite analyser les cellules communes entre les lots.

Voici une explication des différentes méthodes d’intégration disponibles dans IntegrateLayers() :

1️⃣ CCAIntegration (Canonical Correlation Analysis)
	•	Fonctionnement :
CCA est une méthode classique d’intégration qui trouve les relations linéaires entre les variables de deux ensembles de données. Elle est basée sur l’idée que les cellules partagées entre les lots doivent avoir des profils d’expression similaires. CCA optimise l’alignement entre ces ensembles de données pour les intégrer dans un espace de faible dimension.
	•	Quand l’utiliser :
Utile pour des données de transcription (scRNA-seq) provenant de différents lots ou conditions, lorsque l’on s’attend à ce que les relations entre les variables (gènes) soient linéaires.

2️⃣ RPCAIntegration (Regularized Principal Component Analysis)
	•	Fonctionnement :
RPCA est une extension de l’analyse en composantes principales (PCA) qui applique une régularisation pour mieux gérer les variations inter-lots tout en conservant les structures sous-jacentes communes. Elle permet de réduire la variance non désirée entre les différents jeux de données tout en extrayant des composantes principales partagées.
	•	Quand l’utiliser :
Pratique lorsque les jeux de données ont des structures complexes ou des bruits, mais que vous voulez toujours intégrer les informations dans un espace réduit tout en réduisant les effets du batch.

3️⃣ HarmonyIntegration (Harmony)
	•	Fonctionnement :
Harmony est une méthode d’intégration plus récente qui ajuste les scores des composantes principales (PCs) de manière à corriger les effets de batch tout en maintenant les relations biologiques entre les cellules. Harmony ajuste progressivement les composantes principales en fonction des variations de batch tout en préservant les structures biologiques communes.
	•	Quand l’utiliser :
Idéal pour les données de scRNA-seq provenant de plusieurs lots ou de différentes sources, car elle corrige efficacement les effets de batch tout en permettant une intégration biologique cohérente.

4️⃣ FastMNNIntegration (Fast Mutual Nearest Neighbors)
	•	Fonctionnement :
FastMNN est une approche rapide qui repose sur l’identification des plus proches voisins mutuels (MNN) entre différents jeux de données. Elle est plus rapide que les autres méthodes d’intégration basées sur l’optimisation de matrices, et elle est conçue pour intégrer des jeux de données volumineux tout en tenant compte des relations locales entre les cellules.
	•	Quand l’utiliser :
Pratique pour les jeux de données volumineux où il est essentiel de corriger rapidement les effets de batch tout en gardant une structure biologique fine.

5️⃣ scVIIntegration (scVI)
	•	Fonctionnement :
scVI utilise des modèles variational autoencoders (VAE) pour modéliser les données de scRNA-seq sous forme probabiliste. Cette méthode permet une intégration robuste, même avec des données bruyantes, en apprenant une représentation latente des données. Elle corrige automatiquement les variations de batch et préserve les structures biologiques sous-jacentes.
	•	Quand l’utiliser :
À utiliser lorsque vous travaillez avec des données de scRNA-seq complexes et que vous souhaitez une intégration robuste qui tient compte des relations probabilistes sous-jacentes des données.

⸻

Exemple d’utilisation :

Voici comment tu peux utiliser IntegrateLayers() avec l’une de ces méthodes :

integrated_object <- IntegrateLayers(
  object = your_seurat_object, 
  method = "HarmonyIntegration", 
  features = VariableFeatures(your_seurat_object), 
  dims = 1:20, 
  verbose = TRUE
)

	•	method : Choisis l’une des méthodes d’intégration disponibles : "CCAIntegration", "RPCAIntegration", "HarmonyIntegration", "FastMNNIntegration", "scVIIntegration".
	•	features : Utilise les gènes les plus variables (ou toute autre liste de gènes) pour l’intégration.
	•	dims : Choisis le nombre de dimensions principales à utiliser pour l’intégration (par exemple, 1:20).

Cela va créer un objet Seurat intégré avec une réduction dimensionnelle ajustée pour intégrer les données provenant de différents lots ou expériences.

Enfin, les utilisateurs peuvent également effectuer une intégration à l'aide de données normalisées par sctransform (voir notre vignette SCTransform pour plus d'informations), en exécutant d'abord la normalisation SCTransform, puis en définissant l' normalization.methodargument dans IntegrateLayers.

L’intégration avec des données normalisées par SCTransform dans Seurat permet de tirer parti de la normalisation avancée qui corrige à la fois les biais techniques et les effets de batch tout en préservant les caractéristiques biologiques des données. Cette approche est particulièrement utile pour les données de scRNA-seq, car elle permet d’intégrer de manière plus robuste les données provenant de différents lots ou conditions expérimentales.

Étapes d’intégration avec SCTransform :
	1.	Normalisation SCTransform
Avant d’intégrer les données, tu normalises chaque jeu de données en utilisant SCTransform(). Cette méthode ajuste les données de comptage brutes en prenant en compte les biais techniques comme la taille de la bibliothèque, les effets de batch et d’autres facteurs confondants. Elle effectue une transformation basée sur une régression, et applique une normalisation en utilisant un modèle de distribution de type negative binomial.
Exemple de normalisation avec SCTransform :

# Appliquer SCTransform pour chaque objet Seurat
object <- SCTransform(object, verbose = FALSE)


	2.	Utilisation de IntegrateLayers avec SCTransform
Après avoir effectué la normalisation SCTransform, tu peux utiliser la fonction IntegrateLayers() pour intégrer ces données normalisées entre plusieurs lots ou expériences. Le paramètre normalization.method permet de spécifier que les données normalisées par SCTransform doivent être utilisées pour l’intégration.

Exemple d’usage avec SCTransform et IntegrateLayers :

# Normaliser les objets Seurat avec SCTransform
object1 <- SCTransform(object1, verbose = FALSE)
object2 <- SCTransform(object2, verbose = FALSE)

# Intégrer les objets après SCTransform
integrated_object <- IntegrateLayers(
  object = list(object1, object2), 
  normalization.method = "SCT",   # Spécifier que SCTransform est utilisé
  method = "HarmonyIntegration",   # Choisir la méthode d'intégration (par exemple Harmony)
  features = VariableFeatures(object1), 
  dims = 1:20, 
  verbose = TRUE
)

Explication des paramètres utilisés :
	•	normalization.method = "SCT" : Ce paramètre indique que l’intégration se fait sur des données qui ont été normalisées avec SCTransform. Cela permet à Seurat de gérer correctement les effets techniques et les variations entre les lots avant l’intégration.
	•	method = "HarmonyIntegration" : C’est la méthode d’intégration choisie. Ici, on utilise Harmony, mais tu peux aussi utiliser CCA, RPCA, etc.
	•	features = VariableFeatures(object1) : Cette fonction permet de sélectionner les gènes les plus variables de ton objet Seurat pour l’intégration.
	•	dims = 1:20 : Ce paramètre spécifie les premières dimensions principales à utiliser pour l’intégration (par exemple, les 20 premières composantes principales).

Pourquoi utiliser SCTransform pour l’intégration ?

L’utilisation de SCTransform avant l’intégration a plusieurs avantages :
	•	Correction des effets techniques : SCTransform ajuste les données pour éliminer les effets indésirables liés à la taille de la bibliothèque ou aux variations techniques.
	•	Amélioration de la robustesse : La normalisation SCTransform améliore la détection des relations biologiques sous-jacentes tout en réduisant le bruit.
	•	Précision de l’intégration : Lorsque les données sont normalisées via SCTransform, l’intégration devient plus précise, surtout quand il y a des variations importantes entre les lots ou conditions expérimentales.

Conclusion :

Lorsque tu as plusieurs jeux de données de scRNA-seq provenant de différents lots ou conditions, tu peux d’abord normaliser chaque jeu de données avec SCTransform, puis utiliser IntegrateLayers() avec l’argument normalization.method = "SCT" pour intégrer ces données de manière robuste tout en corrigeant les effets de batch. Cela permet d’aligner les données tout en préservant les caractéristiques biologiques communes entre les jeux de données.

Avec Seurat v5, l’intégration de données normalisées par SCTransform a été simplifiée grâce à l’utilisation de IntegrateLayers(). Ce processus est particulièrement puissant pour gérer les effets de batch et intégrer des jeux de données provenant de différentes sources tout en préservant les structures biologiques communes.

Voici comment fonctionne cette intégration dans Seurat v5 :

Étapes pour intégrer des données normalisées avec SCTransform dans Seurat v5

1. Normalisation avec SCTransform

Avant d’effectuer l’intégration, tu dois d’abord normaliser chaque objet Seurat avec la fonction SCTransform(). Cela permet de corriger les effets techniques (comme la taille de la bibliothèque ou la variabilité entre les lots) tout en conservant les informations biologiques importantes.

Exemple de normalisation avec SCTransform :

# Appliquer SCTransform à chaque objet Seurat
object1 <- SCTransform(object1, verbose = FALSE)
object2 <- SCTransform(object2, verbose = FALSE)

2. Utilisation de IntegrateLayers avec SCTransform

Une fois les données normalisées par SCTransform, tu peux utiliser IntegrateLayers() pour intégrer ces jeux de données tout en spécifiant que l’intégration se fait sur des données SCTransform.

Dans Seurat v5, tu dois spécifier l’option normalization.method = "SCT" dans IntegrateLayers() pour indiquer que l’intégration se fait sur des données normalisées via SCTransform.

Exemple d’intégration après normalisation SCTransform :

# Intégration des objets après SCTransform
integrated_object <- IntegrateLayers(
  object = list(object1, object2),         # Liste des objets Seurat à intégrer
  normalization.method = "SCT",            # Indiquer que SCTransform a été utilisé
  method = "HarmonyIntegration",           # Choisir la méthode d'intégration (ex: Harmony)
  features = VariableFeatures(object1),    # Gènes les plus variables à utiliser pour l'intégration
  dims = 1:20,                            # Dimensions principales à utiliser pour l'intégration
  verbose = TRUE                           # Affichage de la progression
)

Explication des principaux arguments dans IntegrateLayers :
	•	object = list(object1, object2) : La liste des objets Seurat à intégrer. Tu peux ajouter autant d’objets que nécessaire (par exemple, deux objets provenant de différents lots ou conditions expérimentales).
	•	normalization.method = "SCT" : Cet argument spécifie que les objets ont été normalisés avec SCTransform. Cela permet à Seurat de gérer correctement les données et d’effectuer une intégration basée sur ces données normalisées.
	•	method = "HarmonyIntegration" : Ici, on utilise la méthode Harmony pour l’intégration. Tu peux aussi utiliser d’autres méthodes comme CCA, RPCA, etc., selon tes besoins.
	•	features = VariableFeatures(object1) : Cela permet de spécifier quels gènes utiliser pour l’intégration. En général, on utilise les gènes les plus variables entre les différents lots pour obtenir une meilleure intégration.
	•	dims = 1:20 : Ce paramètre sélectionne les 20 premières composantes principales à utiliser pour l’intégration. En général, tu veux utiliser les dimensions les plus informatives.
	•	verbose = TRUE : Affiche une barre de progression pour suivre l’avancement de l’intégration.

Avantages de l’intégration avec SCTransform :
	•	Correction des effets de batch : SCTransform applique une normalisation avancée qui permet de réduire les variations dues aux effets techniques (comme la taille de la bibliothèque ou les différences entre les lots).
	•	Amélioration des résultats d’intégration : En utilisant SCTransform, tu t’assures que l’intégration se fait sur des données déjà corrigées pour les biais techniques, ce qui améliore la qualité de l’intégration et des résultats finaux.
	•	Réduction du bruit : SCTransform aide également à réduire le bruit dans les données de scRNA-seq, ce qui rend l’intégration plus robuste et précise.

Conclusion

Avec Seurat v5, l’intégration de données après normalisation SCTransform est simplifiée grâce à IntegrateLayers(), en spécifiant normalization.method = "SCT". Cette approche permet de corriger efficacement les effets de batch tout en préservant les structures biologiques importantes, rendant l’analyse plus robuste et fiable, surtout lorsque les jeux de données proviennent de différentes conditions expérimentales ou lots.

In [ ]:
Explication de la gestion des couches (layers) et de la fusion d’objets dans Seurat v5

Seurat v5 introduit une nouvelle approche pour gérer les échantillons multiples dans un seul objet Seurat à l’aide des couches (layers). Cette approche simplifie l’intégration et l’analyse multi-échantillons.

⸻

🔹 1. Utilisation des couches (Layers) pour gérer plusieurs échantillons

Dans Seurat v5, il est possible de séparer un objet en différentes couches en fonction d’une variable de métadonnées, tout en conservant les données d’expression dans un seul objet.

📌 Pourquoi utiliser les couches ?

✅ Permet de garder tous les échantillons dans un seul objet Seurat, tout en les séparant pour une analyse comparative.
✅ Simplifie le workflow d’intégration car les couches peuvent être directement intégrées.
✅ Permet de fusionner les couches après l’intégration pour retrouver un objet unique.

📍 Exemple d’utilisation

➤ Diviser un objet en couches selon une condition expérimentale (stim)

# Séparer l'assay "RNA" en couches basées sur la condition de stimulation ("stim")
ifnb[["RNA"]] <- split(ifnb[["RNA"]], f = ifnb$stim)

# Vérifier les couches disponibles
Layers(ifnb)

👉 Ici, l’objet Seurat ifnb contient maintenant plusieurs couches : une pour chaque valeur unique de stim (ex. CTRL et STIM).

➤ Fusionner les couches après intégration

# Réunir les couches en un seul objet
ifnb <- JoinLayers(ifnb)

👉 Cette commande reconstitue l’objet original en fusionnant toutes les couches.

⸻

🔹 2. Diviser un objet en plusieurs objets (SplitObject)

Une alternative aux couches consiste à diviser un objet Seurat en plusieurs objets distincts, basés sur une colonne des métadonnées.

📍 Exemple d’utilisation

# Diviser l'objet Seurat en une liste de plusieurs objets selon la condition "stim"
ifnb_list <- SplitObject(ifnb, split.by = "stim")

# Accéder aux objets séparés
ifnb_list$CTRL  # Objet contenant uniquement les cellules "CTRL"
ifnb_list$STIM  # Objet contenant uniquement les cellules "STIM"

👉 Chaque élément de ifnb_list est un objet Seurat distinct, contenant uniquement les cellules d’une condition donnée.

⸻

🔹 3. Fusionner plusieurs objets Seurat (sans intégration)

Si l’on veut fusionner plusieurs objets Seurat sans effectuer d’intégration, Seurat v5 permet de les fusionner dans un même objet, tout en conservant l’information dans des couches séparées.

📍 Exemple d’utilisation

# Fusionner deux objets Seurat (CTRL et STIM)
merged_obj <- merge(x = ifnb_list$CTRL, y = ifnb_list$STIM)

# Réunir les couches après la fusion
merged_obj[["RNA"]] <- JoinLayers(merged_obj)

👉 La fusion regroupe les données dans un même objet, mais garde les échantillons sous forme de couches distinctes, sauf si on utilise JoinLayers() pour tout rassembler.

⸻

🔹 4. Fusionner plusieurs objets avec intégration

Si on souhaite intégrer plusieurs objets pour corriger les effets de batch, on doit d’abord normaliser les données, identifier les features variables, et exécuter une intégration avant de fusionner les couches.

📍 Exemple d’intégration avec RPCAIntegration

# Fusionner les objets
merged_obj <- merge(x = ifnb_list$CTRL, y = ifnb_list$STIM)

# Pré-traitement
merged_obj <- NormalizeData(merged_obj)
merged_obj <- FindVariableFeatures(merged_obj)
merged_obj <- ScaleData(merged_obj)
merged_obj <- RunPCA(merged_obj)

# Intégration des couches avec RPCA
merged_obj <- IntegrateLayers(object = merged_obj, 
                              method = RPCAIntegration, 
                              orig.reduction = "pca", 
                              new.reduction = "integrated.rpca",
                              verbose = FALSE)

# Réunir les couches après l'intégration
merged_obj[["RNA"]] <- JoinLayers(merged_obj)

👉 Ce workflow permet d’intégrer plusieurs échantillons tout en corrigeant les effets de batch.

⸻

Résumé 📌

Méthode	Objectif	Commande principale
Diviser un objet en couches (Layers)	Gérer plusieurs échantillons dans un seul objet	split(ifnb[["RNA"]], f = ifnb$stim)
Fusionner les couches	Réunir toutes les couches en un seul objet	JoinLayers(ifnb)
Diviser un objet en plusieurs objets (SplitObject)	Créer une liste d’objets Seurat distincts	SplitObject(ifnb, split.by = "stim")
Fusionner plusieurs objets sans intégration	Combiner plusieurs objets en un seul sans correction de batch	merge(x = obj1, y = obj2)
Fusionner plusieurs objets avec intégration	Intégrer plusieurs objets avec correction de batch	IntegrateLayers(object, method = RPCAIntegration)



⸻

Quand utiliser quelle méthode ?

🔹 Si tu veux travailler avec plusieurs échantillons sans les séparer totalement, utilise les couches (Layers).
🔹 Si tu veux analyser les échantillons séparément, utilise SplitObject() pour créer des objets distincts.
🔹 Si tu veux fusionner des objets sans correction de batch, utilise merge() et JoinLayers().
🔹 Si tu veux corriger les effets de batch avant fusion, utilise IntegrateLayers() avec une méthode d’intégration.

Cette approche permet une gestion plus flexible et efficace des données multi-échantillons dans Seurat v5 🚀.